In [1]:
from model_functions import analyze_token_sequence, predict, write_midi, get_token_flags
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
from transformers import GPT2Config, GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, TrainingArguments, Trainer

PATH_VOCAB = "../0_data/5_vocabs"
PATH_MODELS = "../0_data/7_models"
PATH_MODELS_CONFIG = "../0_data/7_models/config"
PATH_PRED = "../0_data/8_predictions"
PATH_TOKENS = "../0_data/8_predictions/tokens"
PATH_MIDI = "../0_data/8_predictions/midi"

for path in [PATH_PRED, PATH_TOKENS, PATH_MIDI]:
    if not os.path.exists(path):
        os.makedirs(path)

In [2]:
vocab_configs = {
    "a1" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "a2" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": True,
    },
    "a3" : {
        "pitch_range": 128,
        "duration_steps": 32,
        "triole_tokens": False,
    },
    "b" : {
        "pitch_range": 128,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "c" : {
        "pitch_range": 36,
        "duration_steps": 64,
        "triole_tokens": False,
    },
    "d" : {
        "pitch_range": 36,
        "duration_steps": 32,
        "triole_tokens": True,
    }
}

In [3]:
model_df = pd.read_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx", index_col="Unnamed: 0")
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1_50,1024,256,4,6,0,0.001,50,4,yes,633.0726,10.55,1.341614,33,400.472565,0.21,321.04,1
1,a2_50,1024,256,4,6,0,0.001,50,4,yes,660.1042,11.00,1.133472,41,206.331863,0.25,285.16,1
2,a3_50,1024,256,4,6,0,0.001,50,4,yes,636.4971,10.61,1.348449,36,167.751266,0.25,319.93,1
3,b_50,1024,256,4,6,0,0.001,50,4,yes,636.4864,10.61,1.349513,36,254.352493,0.40,321.09,1
4,c_50,1024,256,4,6,0,0.001,50,4,yes,669.2868,11.15,1.340456,36,163.397003,0.28,320.76,1
5,d_50,1024,256,4,6,0,0.001,50,4,yes,655.0100,10.92,1.167735,50,486.180145,0.34,278.31,1
6,a1,1024,256,4,6,0,0.001,20,4,yes,265.2580,4.42,1.345365,20,137.914551,0.45,321.32,1
7,a2,1024,256,4,6,0,0.001,20,4,yes,260.7209,4.35,1.202786,20,120.460831,0.24,280.73,1
8,a3,1024,256,4,6,0,0.001,20,4,yes,262.6342,4.38,1.377196,20,177.487762,0.35,321.58,1
9,b,1024,256,4,6,0,0.001,20,4,yes,265.6575,4.43,1.343577,20,188.348267,0.27,322.53,1


In [4]:
TICKS_PER_BEAT = 1024
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/32
    
for index, row in model_df.iterrows():

    # only models that ran
    if row["ran"] == "no" or pd.notnull(row["incorrect_notes"]) or row["runtime"] == "too big - cuda error":
        continue
    
    model_name = row["name"]
    model_vocab = model_name.split("_")[0]
    
    if not os.path.exists(f"{PATH_MIDI}/{model_name}"):
        os.makedirs(f"{PATH_MIDI}/{model_name}")

    # get token flags and duration bins
    token_flags = get_token_flags(vocab_configs[model_vocab])
    duration_steps = vocab_configs[model_vocab]["duration_steps"]
    duration_bins = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*duration_steps)+1, TICKS_PER_MIN_DURATION, dtype=int)

    # create tokenizer
    tokenizer = GPT2Tokenizer(
        vocab_file=f"{PATH_VOCAB}/vocab_{model_vocab}.json", 
        merges_file=f"{PATH_VOCAB}/merges.txt")
    tokenizer.add_special_tokens({'pad_token': 'PAD', 'bos_token': 'BOS', 'eos_token': 'EOS',})
    
    # get vocabulary
    with open(f"{PATH_VOCAB}/vocab_{model_vocab}.json", "r") as fp:
        vocab = json.load(fp)
    token2word = {token: word for word, token in vocab.items()}

    # load model
    model = GPT2LMHeadModel.from_pretrained(f"{PATH_MODELS_CONFIG}/{model_name}/end_version")
    
    # make predictions save
    output = predict(model, tokenizer, samples=100, max_length=row["max_length"])
    data_generated = {"data": output}
    
    with open(f"{PATH_TOKENS}/{model_name}.json", "w") as fp:
        json.dump(data_generated, fp)
    
    # analyze tokens and save as midi_files
    correct_notes = 0
    incorrect_notes = 0
    for idx, pred in enumerate(output):
        an = analyze_token_sequence(pred, token_flags)
        correct_notes += an["start-pos-pitch-duration"] + an["start-pos-pitch-duration-dtriole"] + an["start-pos-ptriole-pitch-duration"] + an["start-pos-ptriole-pitch-duration-dtriole"]
        incorrect_notes += write_midi(output[idx], token2word, duration_bins, f"{PATH_MIDI}/{model_name}/generated_midi_{idx}.midi")
    
    model_df.at[index,"correct_notes"] = (correct_notes/100).__round__(2)
    model_df.at[index,"incorrect_notes"] = (incorrect_notes/100).__round__(2)
    model_df.at[index,"correct_rate"] = (correct_notes/(correct_notes+incorrect_notes)).__round__(2)
    
model_df

,name,max_length,emb_dim,attention_heads,layers,dropout,learning_rate,epochs,batch_size,ran,runtime,runtime_min,min_loss,at_epoch,perplexity,incorrect_notes,correct_notes,correct_rate
0,a1_50,1024,256,4,6,0,0.001,50,4,yes,633.0726,10.55,1.341614,33,400.472565,0.21,321.04,1
1,a2_50,1024,256,4,6,0,0.001,50,4,yes,660.1042,11.00,1.133472,41,206.331863,0.25,285.16,1
2,a3_50,1024,256,4,6,0,0.001,50,4,yes,636.4971,10.61,1.348449,36,167.751266,0.25,319.93,1
3,b_50,1024,256,4,6,0,0.001,50,4,yes,636.4864,10.61,1.349513,36,254.352493,0.40,321.09,1
4,c_50,1024,256,4,6,0,0.001,50,4,yes,669.2868,11.15,1.340456,36,163.397003,0.28,320.76,1
5,d_50,1024,256,4,6,0,0.001,50,4,yes,655.0100,10.92,1.167735,50,486.180145,0.34,278.31,1
6,a1,1024,256,4,6,0,0.001,20,4,yes,265.2580,4.42,1.345365,20,137.914551,0.45,321.32,1
7,a2,1024,256,4,6,0,0.001,20,4,yes,260.7209,4.35,1.202786,20,120.460831,0.24,280.73,1
8,a3,1024,256,4,6,0,0.001,20,4,yes,262.6342,4.38,1.377196,20,177.487762,0.35,321.58,1
9,b,1024,256,4,6,0,0.001,20,4,yes,265.6575,4.43,1.343577,20,188.348267,0.27,322.53,1


In [5]:
model_df.to_excel(f"{PATH_MODELS}/comparison_model_stats.xlsx")

run "tar chvfz predictions_midi.tar.gz *" in terminal midi folder to create and download zip